In [ ]:
with open('failory_india_startups.html','r',encoding='utf-8') as f:
  html=f.read()

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup=BeautifulSoup(html,'lxml')

In [ ]:
soup.prettify()

'<html class="w-mod-js wf-opensans-n8-active wf-opensans-n4-active wf-opensans-n7-active wf-opensans-n6-active wf-opensans-n3-active w-mod-ix wf-opensans-i4-active wf-opensans-i7-active wf-opensans-i6-active wf-opensans-i3-active wf-opensans-i8-active wf-active" data-wf-collection="60fb2741bd04787f525dbdcc" data-wf-domain="www.failory.com" data-wf-item-slug="india" data-wf-page="60fb2741bd04782b495dbd7d" data-wf-site="5fada95bd7a18db677087e20" lang="en">\n <head>\n  <style>\n   .wf-force-outline-none[tabindex="-1"]:focus{outline:none;}\n  </style>\n  <meta charset="utf-8"/>\n  <title>\n   Top 300 Startups in India in 2024\n  </title>\n  <meta content="Here\'s an extensive list with the 300 best startups in India and their main information." name="description"/>\n  <meta content="Top 300 Startups in India in 2024" property="og:title"/>\n  <meta content="Here\'s an extensive list with the 300 best startups in India and their main information." property="og:description"/>\n  <meta content

In [ ]:
import re
names=[]
pattern=re.compile(r'^\d+\)')

h3_tags = soup.find_all('h3', string=pattern)


In [ ]:
h3_tags[299]

<h3>300) Fyllo</h3>

In [ ]:
import re
import pandas as pd

# 1. Extract startup names
startup_name = []
pattern = re.compile(r'^\d+\)')
h3_tags = soup.find_all('h3', string=pattern)

for h3 in h3_tags:
    name = re.sub(r'^\d+\)\s*', '', h3.text.strip())
    startup_name.append(name)

# 2. Extract abouts
abouts = []
for h3 in h3_tags:
    p_tag = h3.find_next_sibling('p')
    abouts.append(p_tag.text.strip() if p_tag else "N/A")

# 3. Extract list-based fields
containers = soup.find_all('ul', {'role': 'list'})

states = []
cities = []
years_started = []
founders = []
industries = []
employees = []
total_funding = []
funding_stages = []
investors_count = []

for container in containers:
    s = c = y = f = ind = emp = fund = stage = inv = "N/A"

    for li in container.find_all('li'):
        text = li.get_text().strip().lower()

        if 'state' in text:
            s = li.get_text().strip()
        elif 'city' in text:
            c = li.get_text().strip()
        elif 'started' in text:
            y = li.get_text().strip()
        elif 'founder' in text:
            f = li.get_text().strip()
        elif 'industries' in text:
            ind = li.get_text().strip()
        elif 'number of employees' in text:
            emp = li.get_text().strip()
        elif 'funding:' in text and 'round' not in text:
            fund = li.get_text().strip()
        elif 'funding rounds' in text:
            stage = li.get_text().strip()
        elif 'number of investors' in text:
            inv = li.get_text().strip()

    states.append(s)
    cities.append(c)
    years_started.append(y)
    founders.append(f)
    industries.append(ind)
    employees.append(emp)
    total_funding.append(fund)
    funding_stages.append(stage)
    investors_count.append(inv)

# 4. Fix list lengths
main_len = len(startup_name)

def fix(lst):
    return lst + ["N/A"] * (main_len - len(lst)) if len(lst) < main_len else lst[:main_len]

abouts = fix(abouts)
states = fix(states)
cities = fix(cities)
years_started = fix(years_started)
founders = fix(founders)
industries = fix(industries)
employees = fix(employees)
total_funding = fix(total_funding)
funding_stages = fix(funding_stages)
investors_count = fix(investors_count)

# 5. Create DataFrame
df = pd.DataFrame({
    "Name": startup_name,
    "Description": abouts,
    "State/Province": states,
    "City": cities,
    "Year Started": years_started,
    "Founder(s)": founders,
    "Sector / Industry": industries,
    "Employees": employees,
    "Total Funding": total_funding,
    "Funding Stage": funding_stages,
    "Number of Investors": investors_count
})

# 6. Save to CSV
df.to_csv("failory_startups_pretty.csv", index=False)




            Name                                        Description  \
0  Urban Company  Urban is a marketplace for independent contrac...   
1      Classplus  Classplus is a mobile-first SaaS platform that...   
2          Paytm  Paytm is a payment gateway that allows users a...   
3           Apna  Apna is a professional networking and job-sear...   
4       Razorpay  Razorpay is a payment acceptance, processing, ...   

         State/Province             City      Year Started  \
0        State: Haryana    City: Gurgaon  Started in: 2014   
1  State: Uttar Pradesh      City: Noida  Started in: 2018   
2  State: Uttar Pradesh      City: Noida  Started in: 2010   
3    State: Maharashtra     City: Mumbai  Started in: 2019   
4      State: Karnataka  City: Bengaluru  Started in: 2014   

                                          Founder(s)  \
0  Founders: Abhiraj Singh Bhal, Raghav Chandra, ...   
1  Founders: Bhaswat Agarwal, Bikash Dash, Mukul ...   
2      Founders: Akshay Khanna, 

In [ ]:
df.shape

(300, 11)